In [43]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import random
from pandas import ExcelWriter

# Find 10 interesting conversations

In [2]:
talk = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/talk_filtered_3.csv')
talk

,contribution_id,article,thread,timestamp,username,thread_type,text
0,1,"""Tiger"" William Dunlop",Requested move,2010-07-21 17:13:40,Billinghurst,THREAD_STARTER,The following discussion is an archived discus...
1,2,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlo..."
2,3,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,User:CJ3370/William_Tiger_Dunlop → William Tig...
3,4,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,I'd like to make this contribution to Tiger Du...
4,5,"""Tiger"" William Dunlop",Requested move,2010-07-14 15:12:03,Skinsmoke,POST,Oppose This will result in two articles on th...
5,6,"""Tiger"" William Dunlop",Requested move,2010-07-14 16:59:04,Deconstructhis,POST,Not a problem to merge if that is the most app...
6,7,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,"Comment Sorry that you took offense, but it w..."
7,8,"""Tiger"" William Dunlop",Requested move,2010-07-21 17:13:40,Billinghurst,POST,The above discussion is preserved as an archiv...
8,9,"""Tiger"" William Dunlop",Current image,2010-07-15 20:19:17,Deconstructhis,THREAD_STARTER,I've noticed that the current image that's acc...
9,10,"""Tiger"" William Dunlop",Current image,2010-07-15 20:30:15,Ukexpat,POST,I removed the deletion tag because the image i...


In [3]:
thread_lens = talk.groupby(['article', 'thread']).size()
thread_lens

article                        thread                                                  
"Tiger" William Dunlop         Current image                                                5
                               Requested move                                               8
"V" Is for Vagina              Why is the "i" capitalized?                                 10
&                              "O" as a word                                                4
                               And per se and as a letter                                   5
                               My use                                                       6
                               Programming languages                                        4
                               Usage                                                        3
                               Wikipedia:Manual of Style (And vs. &)                        5
'Abd al-Ilah                   Arabic spelling                    

In [20]:
thresholds = [5, 10, 15, 20, 30, 40]
binned_threads = {min_len: list() for min_len in thresholds}

for min_len in thresholds:
    binned_threads[min_len] = thread_lens[thread_lens > min_len].index.tolist()

binned_threads

{5: [('"Tiger" William Dunlop', 'Requested move'),
  ('"V" Is for Vagina', 'Why is the "i" capitalized?'),
  ('&', 'My use'),
  ("(I Can't Get No) Satisfaction", 'Britney Spears cover'),
  ('.hack//Outbreak', 'Lead image'),
  ('.hack//Outbreak', 'Removed text'),
  ('.tib files', 'Merger proposal'),
  ('11:00PM-12:00AM (24 season 6)',
   'DO NOT EDIT THIS PAGE WITHOUT HARD FACTS'),
  ('11:00PM-12:00AM (24 season 6)', 'Date Error'),
  ('11:00PM-12:00AM (24 season 6)', 'Death Count'),
  ('11:00PM-12:00AM (24 season 6)', 'The episodes have not aired yet on TV.'),
  ('1587 Yellow River flood', 'Length'),
  ('1587 Yellow River flood', 'Mongolian name'),
  ('17-Mile Drive', '17 Mile Drive info box and section images replacement'),
  ('17-Mile Drive', 'RFC on Lead Image'),
  ('17-Mile Drive', 'Set C'),
  ('180 Solutions', '2014 Neutrality discussion'),
  ('180 Solutions', 'An update on the spyware status?'),
  ('180 Solutions', 'Private Message'),
  ('180 Solutions', "Yes, it's spyware"),
  ('

In [6]:
for min_len in thresholds:
    print(min_len, end=': ')
    print(len(binned_threads[min_len]))

10: 2164
15: 1090
20: 617
30: 220
40: 101


In [8]:
# Sample from 15-threshold threads
sample = random.sample(binned_threads[15], 10)
sample

[('Anton Balasingham', 'Asia Tribune vs. Tamilnet'),
 ('Oh Eun-sun', 'Biased Edits'),
 ('Yoni Jesner', 'DYK preparation'),
 ('Greater Assam', 'Other states of India:- Citation needed.'),
 ('Japanese acupuncture', 'Original research in the article again'),
 ('MAO College',
  'Undue weight, duplicative coverage and violation of Summary Style'),
 ('Commodity fetishism',
  'when words are scarce they are seldom spent in vein'),
 ('Belgic people', 'Geography'),
 ('Einstein', 'Including relevant quotes'),
 ('E-101 Beta',
  'RfC: Do the characters discussed here meet WP:N such that they should be allowed to keep their articles?')]

In [14]:
# Sample from 10-threshold threads
samples = {}
samples[10] = random.sample(binned_threads[10], 10)
samples[10]

[('Polish occupation of Czechoslovakia',
  'Extent of coverage of the postwar expulsions'),
 ('Priesthood keys', 'Priesthood (Mormonism) Chart'),
 ('US Airways Marketplace', 'Twitter Pornographic Incident'),
 ('Acharya S', 'Delete actual name'),
 ('Eric Harris and Dylan Klebold', 'Marilyn Manson/Misleading use of quote'),
 ('Sandra Caron', '"Non-free" image use'),
 ('Satish Dhawan Space Centre', 'Why 4 articles?'),
 ('Neutral wire', 'Is  Neutral a Current-Carrying Conductor?'),
 ('Kiev Pechersk Lavra', 'English name with "caves"'),
 ('Late Imperial China', 'Modern history')]

In [22]:
# Sample from 5-threshold threads
t = 5
samples = {}
samples[t] = random.sample(binned_threads[t], 10)
samples[t]

[('Rubber tired metro',
  'Rubber Tired Metros as burdensome and having only political (not technical) merit'),
 ('33550336 (number)', 'The first two lines in disagreement.'),
 ('Thermal light', 'Reworked'),
 ('Satan (South Park)', 'Proposed removal of content'),
 ('Mount Hebron', 'Removal of well sourced material'),
 ('Tour de France', 'Armstrong'),
 ('Avant Browser', "Taken from Avant's Site"),
 ('Storyline of the Mario series', 'Appearances'),
 ('Clare Balding', 'Controversy'),
 ('Decimation (comics)', 'The 198 Files')]

In [11]:
pd.set_option('display.max_colwidth', 999)

In [12]:
n = 0
a,t = sample[n]
rows = talk[(talk['article']==a) & (talk['thread']==t)]
rows

# for a,t in sample:
#     rows = talk[(talk['article']==a) & (talk['thread']==t)]
#     print(rows)

,contribution_id,article,thread,timestamp,username,thread_type,text
7614,16900,Anton Balasingham,Asia Tribune vs. Tamilnet,2006-12-26 10:32:42,Rajsingam,THREAD_STARTER,"Once you read Asia Tribune you will come to know where it stands and who are really behind the 'Asia', at least Tamilnet is openly disclosing who are they, that is the difference."
7615,16901,Anton Balasingham,Asia Tribune vs. Tamilnet,2006-12-26 01:26:52,Rajsingam,POST,Rajsingam 26 Dec. 2006
7616,16902,Anton Balasingham,Asia Tribune vs. Tamilnet,2006-12-26 10:36:42,Bakasuprman,POST,"U mean Associated Sinhalese Intelligence Advisors?Bakaman 16:36, 26 December 2006 (UTC)"
7617,16903,Anton Balasingham,Asia Tribune vs. Tamilnet,2006-12-26 10:38:11,Rajsingam,POST,You are correct.
7618,16904,Anton Balasingham,Asia Tribune vs. Tamilnet,2006-12-26 01:26:52,Rajsingam,POST,Rajsingam 26 Dec. 2006
7619,16905,Anton Balasingham,Asia Tribune vs. Tamilnet,2006-12-26 10:44:04,Iwazaki,POST,"nope..You are both disappointingly wrong !! Have you even bother to read ,what the write ?? Even in this case, there are pictures for god sake !! dont blame Asia Tribune for your vision problems --Iwazaki 16:44, 26 December 2006 (UTC)"
7620,16906,Anton Balasingham,Asia Tribune vs. Tamilnet,2006-12-26 10:50:17,Rajsingam,POST,"You go through Tamilnet at times they give prominence to Sri Lankan Government politicians. But the Asia Tribune once heavily attacked the former Singapore Prime Minister Lee Kuan Yew, because he went to say something in his book supporting Sri Lankan Tamils. You do few 'google' search you will come to the shadow of the Asia Tribune."
7621,16907,Anton Balasingham,Asia Tribune vs. Tamilnet,2006-12-26 01:26:52,Rajsingam,POST,Rajsingam 26 Dec. 2006
7622,16908,Anton Balasingham,Asia Tribune vs. Tamilnet,2006-12-26 11:00:08,Iwazaki,POST,"Why dont you provide me the article ? you are the one,who is claiming some thing here, so why dont you give your evidences ?? then i can actually give my comments on it--Iwazaki 17:00, 26 December 2006 (UTC)"
7623,16909,Anton Balasingham,Asia Tribune vs. Tamilnet,2006-12-26 10:52:28,Bakasuprman,POST,"I was kidding about the acronym, but I do find that newspaper full of POV. They might as well throw in a qaseeda for Junius Richard Jayewardene while they're at it. Needless to say that Tamilnet is POV.Bakaman 16:52, 26 December 2006 (UTC)"


# Gather samples into CSV

In [30]:
selected = [
    ('Anton Balasingham', 'Asia Tribune vs. Tamilnet'),
    ('Polish occupation of Czechoslovakia', 'Extent of coverage of the postwar expulsions'),
    ('Priesthood keys', 'Priesthood (Mormonism) Chart'),
    ('Acharya S', 'Delete actual name'),
    ('Rubber tired metro', 'Rubber Tired Metros as burdensome and having only political (not technical) merit'),
    ('33550336 (number)', 'The first two lines in disagreement.'),
    ('Thermal light', 'Reworked'),
    ('Satan (South Park)', 'Proposed removal of content'),
    ('Mount Hebron', 'Removal of well sourced material'),
    ('Tour de France', 'Armstrong')
]
len(selected)

10

In [52]:
# Put in editor scores
scores = pd.read_csv('/home/michael/school/research/wp/wikipedia/data/talk/enwiki/enwiki_talk_scores.csv')
scores.columns

Index(['article', 'thread_title', 'editor', 'editor_talk', 'other_talk',
       '#editor_turns', '#other_turns', 'editor_score'],
      dtype='object')

In [56]:
merged = pd.merge(talk, scores, left_on=['article', 'thread', 'username'], right_on=['article', 'thread_title', 'editor'])
merged = merged[talk.columns.tolist() + ['editor_score']]
merged

,contribution_id,article,thread,timestamp,username,thread_type,text,editor_score
0,2,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlop — Move to provide a more substancial bio of Tiger Dunlop. CJ_WeißSchäfer 16:47, 14 July 2010 (UTC)",0.826415
1,2,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"William ""Tiger"" Dunlop → William ""Tiger"" Dunlop — Move to provide a more substancial bio of Tiger Dunlop. CJ_WeißSchäfer 16:47, 14 July 2010 (UTC)",0.826415
2,4,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"I'd like to make this contribution to Tiger Dunlop's bio.CJ_WeißSchäfer 16:47, 14 July 2010 (UTC)",0.826415
3,4,"""Tiger"" William Dunlop",Requested move,2010-07-14 10:47:31,CJ3370,POST,"I'd like to make this contribution to Tiger Dunlop's bio.CJ_WeißSchäfer 16:47, 14 July 2010 (UTC)",0.826415
4,3,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,User:CJ3370/William_Tiger_Dunlop → William Tiger Dunlop,0.456897
5,3,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,User:CJ3370/William_Tiger_Dunlop → William Tiger Dunlop,0.456897
6,5,"""Tiger"" William Dunlop",Requested move,2010-07-14 15:12:03,Skinsmoke,POST,"Oppose This will result in two articles on the same person, one under William Tiger Dunlop and the other under William ""Tiger"" Dunlop. We do not improve articles by shunting the original off into another title. The way to deal with this is to merge the improvements into the existing article. Skinsmoke (talk) 21:12, 14 July 2010 (UTC)",0.456897
7,5,"""Tiger"" William Dunlop",Requested move,2010-07-14 15:12:03,Skinsmoke,POST,"Oppose This will result in two articles on the same person, one under William Tiger Dunlop and the other under William ""Tiger"" Dunlop. We do not improve articles by shunting the original off into another title. The way to deal with this is to merge the improvements into the existing article. Skinsmoke (talk) 21:12, 14 July 2010 (UTC)",0.456897
8,7,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,"Comment Sorry that you took offense, but it wasn't a ""royal"" we; it's meaning was that we, as Wikipedia editors or the Wikipedia community, don't do it that way. Now to add being patronising to my crimes: you've done an excellent job with the merge. Have you really only been on Wikipedia for a month? Don't worry about getting things wrong every so often. We (and I mean we) are all still learning, and there's not much you can do that can't be undone if a mistake is made. Enjoy your time on here, and if this article is anything to go by, may you be here a long time! Skinsmoke (talk) 01:49, 15 July 2010 (UTC)",0.456897
9,7,"""Tiger"" William Dunlop",Requested move,2010-07-14 19:49:58,Skinsmoke,POST,"Comment Sorry that you took offense, but it wasn't a ""royal"" we; it's meaning was that we, as Wikipedia editors or the Wikipedia community, don't do it that way. Now to add being patronising to my crimes: you've done an excellent job with the merge. Have you really only been on Wikipedia for a month? Don't worry about getting things wrong every so often. We (and I mean we) are all still learning, and there's not much you can do that can't be undone if a mistake is made. Enjoy your time on here, and if this article is anything to go by, may you be here a long time! Skinsmoke (talk) 01:49, 15 July 2010 (UTC)",0.456897


In [60]:
cols = merged.columns.tolist()
new_cols = cols[:3] + ['username', 'timestamp'] + cols[5:]
merged = merged[new_cols]

In [61]:
mask = list(map(lambda tup: tup in selected, zip(merged['article'], merged['thread'])))
selected_rows = merged[mask]
selected_rows

,contribution_id,article,thread,username,timestamp,thread_type,text,editor_score
1308,4095,33550336 (number),The first two lines in disagreement.,CRGreathouse,2006-09-03 19:04:11,POST,"A number always divides itself, but a number's proper divisors don't include itself. Thus if \sigma(n)=2n (where sigma is the number of divisors) the proper divisors are equal to n (since 2n-n=n). Don't worry, it's easy to get confused here with the terminology, even though it's not actually difficult when you get past that. CRGreathouse (t | c) 01:04, 4 September 2006 (UTC)",0.342978
1309,4103,33550336 (number),The first two lines in disagreement.,CRGreathouse,2007-01-01 16:35:18,POST,"You know I don't discuss with you. Your comments are not welcome. 69.158.101.116 21:47, 1 January 2007 (UTC)\n: Anon, please see Wikipedia:Etiquette and WP:NPA. Arthur Rubin made valid points, and you have yet to respond to them. Your notation is non-standard and any proof you present is OR -- and much as I'd like to see a proof, this isn't the place. Now if you put this in your userspace instead of here, it might be better-received, but unless you do that or get a paper published this just isn't the place. I do wish you luck with your proof, though. CRGreathouse (t | c) 22:35, 1 January 2007 (UTC)",0.342978
1310,4104,33550336 (number),The first two lines in disagreement.,CRGreathouse,2011-03-09 20:20:41,POST,"I wonder if there is a published source for the second ""equivalent"" definition, documenting its relevance for being adduced by WP. If not, I would suggest to remove it from this article. --88.78.159.212 (talk) 07:47, 9 March 2011 (UTC)\n:I have now checked it myself and have found it, among others, in B. W. Jones, The theoriy of numbers, 1961, p.57 [ ], and M. J. Bradley, The age of genius, 2006, p.53 [ ]. The latter seems to suggest that it can be found in Fermat. --88.78.159.212 (talk) 08:26, 9 March 2011 (UTC)\n:: They're quite standard; any basic number theory text should have it. Also see the OEIS, MathWorld, etc. CRGreathouse (t | c) 02:20, 10 March 2011 (UTC)",0.342978
1311,4096,33550336 (number),The first two lines in disagreement.,JoshuaZ,2006-09-29 15:39:24,POST,"It is said if an odd perfect number exists, it is of the form N=P4n+1Q2, where P=4m+1 is a prime and Q is a distinct odd prime. Suppose such N is perfect, then 2=[(1+P+P2+...+P4n+1)(1+Q+Q2)]/(P4n+1Q2)",0.267241
1312,4098,33550336 (number),The first two lines in disagreement.,Arthur Rubin,2007-01-01 08:55:36,POST,:(1) 2X(X2)2xY2yZ2z=(1+X)σ[(X2)2xY2yZ2z]; and,0.680794
1313,4100,33550336 (number),The first two lines in disagreement.,Arthur Rubin,2007-01-01 08:55:36,POST,"Suppose X is 1 mod 4. The LHS of (1) is 2 mod 4. Since 1+X on the RHS of (1) is 2 mod 4, σ[…] on the RHS od (1) is 1 mod 4. Now, the LHS of (2) is 1 mod 4. In order for (1+X)/2 on the RHS of (2) to be 1 mod 4, X is 1 mod 8.\nWith X being 1 mod 8, the LHS of (1) is 2 mod 8. Since 1+X on the RHS of (1) is 2 mod 8, σ[…] is 1 mod 8. Now, the LHS of (2) is 1 mod 8. X is 1 mod 16 so that (1+X)/2 is 1 mod 8.\nThat X is 1 mod 16 is more restrictive than X is 1 mod 4.\nAlso, if an OPN exists, the following necessary bounds must be satisfied, with X=16k+1:\n:[(1+X+X2+…+X5)/X5] [(1+Y+Y2)/Y2] [(1+Z+Z2)/Z2] ……\nerrors results. That is uncivil.",0.680794
1314,4101,33550336 (number),The first two lines in disagreement.,Arthur Rubin,2011-02-18 12:23:54,POST,::#Using σ[(X2)2x] to indicate 1 + X2 + ... + X24x. That's just wrong.,0.680794
1315,4102,33550336 (number),The first two lines in disagreement.,Arthur Rubin,2007-01-01 08:55:36,POST,"::#Trying to prove there are no odd perfect numbers in this forum. That would be [WP:OR]], even if you knew what you were doing.\n::— Arthur Rubin | (talk) 14:55, 1 January 2007 (UTC)",0.680794
2027,6395,Acharya S,Delete actual name,RL0919,2013-11-20 04:52:07,THREAD_STARTER,"Acharya S responded on her Facebook page that due to her occupation she feels duress from stalkers knowing her ""actual name."" Per 

In [62]:
selected_rows.to_csv('/home/michael/school/research/wp/jsalt_tutorial/examples.csv', index=False)

In [65]:
example_dfs = [merged[merged['article']==a] for a in [art for art,_ in selected]]
len(example_dfs)

10

In [66]:
writer = ExcelWriter('/home/michael/school/research/wp/jsalt_tutorial/wikipedia_talk_page_examples.xlsx')

for df in example_dfs:
    df.to_excel(writer, df['article'].unique().tolist()[0][:31], index=False)
    
writer.save()